In [23]:
import cv2
from ultralytics import YOLO
from ultralytics.models import YOLOv10

test_image = cv2.imread('test.png')

model_object_detect = YOLO('models/yolov8x-oiv7.pt')
model_face_emmotion = YOLOv10('models/yolov10n-face.pt')

od = model_object_detect(test_image)[0]
fc = model_face_emmotion(test_image)[0]

c:\Users\crazy\.conda\envs\kdt\lib\site-packages\ultralytics\nn\tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")



0: 320x640 1 Boy, 1 Hat, 1 Human face, 1 Man, 1596.3ms
Speed: 2.0ms preprocess, 1596.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 256x512 1 panic, 75.4ms
Speed: 1.0ms preprocess, 75.4ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 512)


In [24]:
import json

# face 모델 라벨
emotion_mapping = {0 : '분노', 1 : '슬픔', 2 : '공포', 3 : '기쁨'}

# oiv7 모델 라벨 JSON 파일에서 딕셔너리 읽기
with open('models/oiv7_jabels.json', 'r') as file:
    oiv7_jabels = json.load(file)

In [40]:
label_fc = [emotion_mapping[int(i)] for i in fc.boxes.cls]
label_od = [oiv7_jabels[str(int(i))] for i in od.boxes.cls]


In [44]:
all_labels = (label_fc + label_od)
exception_lst = ['인간의 얼굴','의류','남자','여자','소년','소녀'] # 텍스트 입력 제외 목록

In [43]:
all_labels

['공포', '인간의 얼굴', '모자', '남자', '소년']

In [45]:
text_intput_text = ''
for i in all_labels:
    if i not in exception_lst:
        text_intput_text +=i + ','

text_intput_text = text_intput_text[:-1]

In [46]:
text_intput_text

'공포,모자'

In [49]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration
import torch

# 저장된 모델과 토크나이저 로드
model_save_path = 'models/t5/model/'
tokenizer_save_path = 'models/t5/model/tokenizer/'

model = T5ForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = T5TokenizerFast.from_pretrained(tokenizer_save_path)

# 테스트 입력
test_input = text_intput_text
# 입력 토큰화
input_ids = tokenizer.encode(test_input, return_tensors='pt')

# 모델 예측
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# 예측 결과 디코딩
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"입력값: {test_input}")
print(f"출력값: {predicted_text}")


테스트 입력: 공포,모자
모델의 예측: 무서운 분위기가 느껴지네요! 😱 어떤 상황인지 궁금해요!


In [51]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 저장된 모델 및 토크나이저 불러오기
model_path = 'models/gpt2/models/'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path + '/tokenizer')

# 평가 모드로 변경
model.eval()

def generate_text(prompt, model, tokenizer, max_length=128, num_return_sequences=1):
    # 입력 텍스트를 토큰화
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # 생성 인자를 설정하여 모델이 텍스트를 생성
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=30,
        top_k=50,
        top_p=0.85,
        temperature=1.7,
        do_sample=True,
        early_stopping=True
    )

    # 생성된 텍스트를 디코딩
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    
    return generated_texts

# 예시: "prompt"에 원하는 문장을 넣어서 결과를 확인
prompt = f"입력값 : {test_input} \n출력값 :"
generated_texts = generate_text(prompt, model, tokenizer)
    
for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:")
    print(text)
    print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1:
입력값 : 공포,모자 
출력값 : 와, 정말 무섭게 느껴지네요... 무슨 일이 있었던 건가요? 그 공포감이 느껴지�

